In [1]:
using EISAnalysis
using Documenter: doctest

In [2]:
doctest(EISAnalysis)

┌ Info: SetupBuildDirectory: setting up build directory.
└ @ Documenter /Users/briandamerau/.julia/packages/Documenter/iwb7N/src/builder_pipeline.jl:75
┌ Info: Doctest: running doctests.
└ @ Documenter /Users/briandamerau/.julia/packages/Documenter/iwb7N/src/builder_pipeline.jl:209
┌ Error: doctest failure in ~/Documents/GitHub/EISAnalysis.jl/src/circuits.jl:17-21
│ 
│ ```jldoctest
│ julia> using EISAnalysis
│ julia> eval(initialize());
│ julia> silly_circuit = 0.25r/2.5l-(r-9wo)/q^0.64-(16r-ws^144)/12.1c;
│ ```
│ 
│ Subexpression:
│ 
│ silly_circuit = 0.25r/2.5l-(r-9wo)/q^0.64-(16r-ws^144)/12.1c;
│ 
│ Evaluated output:
│ 
│ ERROR: UndefVarError: `r` not defined in `Main`
│ Suggestion: check for spelling errors or missing imports.
│ Stacktrace:
│  [1] top-level scope
│    @ none:1
│ 
│ Expected output:
│ 
│ using EISAnalysis
│ 
│   diff = using EISAnalysisERROR: UndefVarError: `r` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Stacktrace:
 [1] top-level

Doctests: EISAnalysis: Test Failed at /Users/briandamerau/.julia/packages/Documenter/iwb7N/src/doctest.jl:110
  Expression: all_doctests()

Stacktrace:
 [1] macro expansion
   @ ~/.julia/juliaup/julia-1.11.3+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Test/src/Test.jl:679 [inlined]
 [2] macro expansion
   @ ~/.julia/packages/Documenter/iwb7N/src/doctest.jl:110 [inlined]
 [3] macro expansion
   @ ~/.julia/juliaup/julia-1.11.3+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Test/src/Test.jl:1704 [inlined]
 [4] doctest(source::String, modules::Vector{Module}; fix::Bool, testset::String, doctestfilters::Vector{Regex}, plugins::Vector{Documenter.Plugin})
   @ Documenter ~/.julia/packages/Documenter/iwb7N/src/doctest.jl:110
Test Summary:         | Fail  Total  Time
Doctests: EISAnalysis |    1      1  2.8s


Test.TestSetException: Some tests did not pass: 0 passed, 1 failed, 0 errored, 0 broken.